In [1]:
import xlwings as xw
import pandas as pd
import requests
import time
wb = xw.Book()
stocks = ["2330" , "4938", "3231","00878", "2883"]
headers = ["股票代號", "最近5年平均合計殖利率%"]
line_token = "your token"

for stock in stocks:
    tables = pd.read_html(f"https://www.stockinfo.tw/?stockSearch={stock}")
    df = tables[1]
    df2 = df.set_index("除權息日")
    new_sheet = wb.sheets.add(name=stock)
    new_sheet.range("A1").value = df2
    new_sheet.range("M1").value = "最近5年平均合計殖利率"
    new_sheet.range("M2").formula = "=AVERAGE(G2:G6)"
new_sheet2 = wb.sheets.add(name="彙整近五年殖利率")
new_sheet2.range("A1").value = headers
j = 2
for i in stocks:
    new_sheet2.range(f"A{j}").value = i
    new_sheet2.range(f"B{j}").value = wb.sheets[f"{i}"].range("M2").value * 100
    j += 1


#DataFrame 匯入台股 現金+股票股利 >= 2.5元 & 目前股價<=100元 csv 數據
df3 = pd.read_csv(r"神秘金字塔 - 股利連續配發排行.csv")

#先計算2020 年的現金殖利率
df3["殖利率%"] = df3["2020"] / df3["收盤價"] * 100

#篩選殖利率高排到低
df4 = df3[["股票", "2020",  "殖利率%", "收盤價"]].sort_values("殖利率%", ascending=False)
new_sheet3 = wb.sheets.add(name="2020殖利率%股票")
new_sheet3.range("A1").value = df4
wb.save("近期觀察股票殖利率彙整表及2020殖利率股票篩選.xlsx")
wb.close()

# Line Notify
line_url = "https://notify-api.line.me/api/notify"

headers = {
    "Authorization": "Bearer " + line_token, 
    "Content-Type" : "application/x-www-form-urlencoded"
}

msg = f"董先生: {time.strftime('%Y/%m/%d')} 您關心的股票殖利率已經更新完畢！"

payload = {'message': msg }
r = requests.post(line_url, headers = headers, params = payload)